In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/8ce5535200b9419d9761e8ffb53b2882



In [1]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
from Model_Unet_github import *

home = os.environ['HOME']

In [2]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_no_intersect_1conv_64_channels_30epoch_unet_github_auto_dimension', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=40, type=int,help='max epoch numbers')



# file paths
#parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
#parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-ckpt_root', default="/root/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/root/datasets", type=str, help=" root to any data folder ")

parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

#experiment.set_name(name)
#experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 40
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/segnet_no_intersect_1conv_64_channels_30epoch_unet_github_auto_dimension
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
[None, 32, 128, 1]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


0 conv1: (?, ?, ?, 32)
0 conv2: (?, ?, ?, 32)
1 conv1: (?, ?, ?, 64)
1 conv2: (?, ?, ?, 64)
2 conv1: (?, ?, ?, 128)
2 conv2: (?, ?, ?, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


...

TRAIN: Batch: 0.574622781643343 Loss: 0.005422748
accuracy = 0.974559
mean IU  = 0.782400
    class # 0 capture rate = 0.974279 
    class # 1 capture rate = 0.981759 
TRAIN: Batch: 0.5785317801579235 Loss: 0.015209844
accuracy = 0.932470
mean IU  = 0.753307
    class # 0 capture rate = 0.927364 
    class # 1 capture rate = 0.980823 
TRAIN: Batch: 0.5824407786725041 Loss: 0.0070879133
accuracy = 0.967325
mean IU  = 0.810115
    class # 0 capture rate = 0.965842 
    class # 1 capture rate = 0.989486 
TRAIN: Batch: 0.5863497771870847 Loss: 0.0139672635
accuracy = 0.954881
mean IU  = 0.776101
    class # 0 capture rate = 0.953520 
    class # 1 capture rate = 0.973039 
TRAIN: Batch: 0.5902587757016652 Loss: 0.010994779
accuracy = 0.957944
mean IU  = 0.797703
    class # 0 capture rate = 0.956096 
    class # 1 capture rate = 0.980223 
TRAIN: Batch: 0.5941677742162458 Loss: 0.0058838255
accuracy = 0.972967
mean IU  = 0.815221
    class # 0 capture rate = 0.972204 
    class # 1 capture r

TRAIN: Batch: 0.7661637088577906 Loss: 0.006152955
accuracy = 0.969995
mean IU  = 0.813850
    class # 0 capture rate = 0.968922 
    class # 1 capture rate = 0.987148 
TRAIN: Batch: 0.7700727073723712 Loss: 0.0052239783
accuracy = 0.976926
mean IU  = 0.844015
    class # 0 capture rate = 0.975855 
    class # 1 capture rate = 0.994480 
TRAIN: Batch: 0.7739817058869518 Loss: 0.006542124
accuracy = 0.972947
mean IU  = 0.833921
    class # 0 capture rate = 0.971838 
    class # 1 capture rate = 0.989502 
TRAIN: Batch: 0.7778907044015323 Loss: 0.007683041
accuracy = 0.973675
mean IU  = 0.836252
    class # 0 capture rate = 0.973645 
    class # 1 capture rate = 0.974121 
TRAIN: Batch: 0.7817997029161129 Loss: 0.008952574
accuracy = 0.964302
mean IU  = 0.789102
    class # 0 capture rate = 0.963456 
    class # 1 capture rate = 0.977907 
TRAIN: Batch: 0.7857087014306935 Loss: 0.004519193
accuracy = 0.983203
mean IU  = 0.860878
    class # 0 capture rate = 0.982993 
    class # 1 capture ra

TRAIN: Batch: 0.9577046360722383 Loss: 0.0056525758
accuracy = 0.975065
mean IU  = 0.847569
    class # 0 capture rate = 0.973899 
    class # 1 capture rate = 0.991763 
TRAIN: Batch: 0.9616136345868188 Loss: 0.0068148626
accuracy = 0.967354
mean IU  = 0.813324
    class # 0 capture rate = 0.965582 
    class # 1 capture rate = 0.993158 
TRAIN: Batch: 0.9655226331013994 Loss: 0.008845981
accuracy = 0.960196
mean IU  = 0.746815
    class # 0 capture rate = 0.959757 
    class # 1 capture rate = 0.969040 
TRAIN: Batch: 0.96943163161598 Loss: 0.0061767465
accuracy = 0.974068
mean IU  = 0.817072
    class # 0 capture rate = 0.973510 
    class # 1 capture rate = 0.984345 
TRAIN: Batch: 0.9733406301305606 Loss: 0.005990437
accuracy = 0.972556
mean IU  = 0.830224
    class # 0 capture rate = 0.971556 
    class # 1 capture rate = 0.987742 
TRAIN: Batch: 0.9772496286451411 Loss: 0.006791024
accuracy = 0.976436
mean IU  = 0.855522
    class # 0 capture rate = 0.975669 
    class # 1 capture ra

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 96.749576%. Class 0 capture: 96.636140%. Class 1 capture: 98.527527%
Character error rate not improved
Done with training at epoch 40 sigoptObservation=0.9683512304002657
